## Setup

In [ ]:
import os,math,sys,torch,re,numpy as np
from types import SimpleNamespace as ns

In [ ]:
np.set_printoptions(precision=2, linewidth=140)
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)

In [ ]:
sys.path.insert(0, '..')

In [ ]:
from utils import show_img,load_cuda,cuda_begin

In [ ]:
%load_ext wurlitzer

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING']='1'
torch.manual_seed(42);

In [ ]:
m1 = torch.rand(50_000, 784)
m1s = m1[:8]
m2 = torch.rand(784,10)

## Reminder

### 2d Python kernel

In [ ]:
def blk_kernel2d(f, blocks, threads, *args):
    for i0 in range(blocks.y):
        for i1 in range(blocks.x):
            for j0 in range(threads.y):
                for j1 in range(threads.x): f(ns(x=i1,y=i0), ns(x=j1,y=j0), threads, *args)

In [ ]:
def matmul_bk(blockidx, threadidx, blockdim, m, n, out, h, w, k):
    r = blockidx.y*blockdim.y + threadidx.y
    c = blockidx.x*blockdim.x + threadidx.x
    
    if (r>=h or c>=w): return
    o = 0.
    for i in range(k): o += m[r*k+i] * n[i*w+c]
    out[r*w+c] = o

In [ ]:
def matmul_2d(m, n):
    h,k  = m.shape
    k2,w = n.shape
    assert k==k2, "Size mismatch!"
    output = torch.zeros(h, w, dtype=m.dtype)
    tpb = ns(x=16,y=16)
    blocks = ns(x=math.ceil(w/tpb.x), y=math.ceil(h/tpb.y))
    blk_kernel2d(matmul_bk, blocks, tpb,
                 m.flatten(), n.flatten(), output.flatten(), h, w, k)
    return output

In [ ]:
torch.isclose(matmul_2d(m1s, m2), m1s@m2).all()

tensor(True)

### CUDA

In [ ]:
cuda_src = cuda_begin + r'''
__global__ void matmul_k(float* m, float* n, float* out, int h, int w, int k) {
    int r = blockIdx.y*blockDim.y + threadIdx.y;
    int c = blockIdx.x*blockDim.x + threadIdx.x;

    if (r>=h || c>=w) return;
    float o = 0;
    for (int i = 0; i<k; ++i) o += m[r*k+i] * n[i*w+c];
    out[r*w+c] = o;
}

torch::Tensor matmul(torch::Tensor m, torch::Tensor n) {
    CHECK_INPUT(m); CHECK_INPUT(n);
    int h = m.size(0);
    int w = n.size(1);
    int k = m.size(1);
    TORCH_CHECK(k==n.size(0), "Size mismatch!");
    auto output = torch::zeros({h, w}, m.options());

    dim3 tpb(16,16);
    dim3 blocks(cdiv(w, tpb.x), cdiv(h, tpb.y));
    matmul_k<<<blocks, tpb>>>(
        m.data_ptr<float>(), n.data_ptr<float>(), output.data_ptr<float>(), h, w, k);
    C10_CUDA_KERNEL_LAUNCH_CHECK();
    return output;
}
'''

In [ ]:
fname = 'matmul'

In [ ]:
def get_sig(fname, src):
    res = re.findall(rf'^(.+\s+{fname}\(.*?\))\s*{{?\s*$', src, re.MULTILINE)
    return res[0]+';' if res else None

In [ ]:
cpp_src = get_sig(fname, cuda_src)
cpp_src

'torch::Tensor matmul(torch::Tensor m, torch::Tensor n);'

In [ ]:
module = load_cuda(cuda_src, cpp_src, [fname])

In [ ]:
m1c,m2c = m1.contiguous().cuda(),m2.contiguous().cuda()

In [ ]:
module.matmul(m1c,m2c).shape

torch.Size([50000, 10])

In [ ]:
torch.isclose(module.matmul(m1c,m2c), m1c@m2c).all()

tensor(True, device='cuda:0')

## Shared mem

### 2d Python kernel

In [ ]:
a = torch.zeros(5)
b,c = a[:3],a[3:]

In [ ]:
b[1] = 2
c[0] = 6
a

tensor([0., 2., 0., 6., 0.])

In [ ]:
def blk_kernel2d_shar(f, blocks, threads, sh_sz, *args, **kwargs):
    for i0 in range(blocks.y):
        for i1 in range(blocks.x):
            shared = torch.zeros(sh_sz)
            f(ns(x=i1,y=i0), threads, shared, *args, **kwargs)

In [ ]:
def run_threads(f, blockdim, *args, **kwargs):
    for i0 in range(blockdim.y):
        for i1 in range(blockdim.x): f(i0, i1, *args, **kwargs)

In [ ]:
def matmul_tiled_bk(blockidx, blockdim, shared, m, n, out, h, w, k, tw):
    shar_sz = tw*tw
    ms,ns = shared[:shar_sz],shared[shar_sz:]

    def get_rc(ty, tx): return blockidx.y*blockdim.y + ty, blockidx.x*blockdim.x + tx

    def fill_shared_tk(ty, tx, ph):
        r,c = get_rc(ty, tx)
        ms[ty*tw+tx] = m[ tx + ph*tw + r*k] if r<h and (ph*tw+tx)<k else 0.
        ns[ty*tw+tx] = n[(ty + ph*tw)*w +c] if c<w and (ph*tw+ty)<k else 0.

    def dotprod_tk(ty, tx):
        r,c = get_rc(ty, tx)
        for i in range(tw):
            if r*w+c<len(out): out[r*w+c] += ms[ty*tw+i] * ns[tw*i+tx]

    for ph in range(int(math.ceil(k/tw))):
        run_threads(fill_shared_tk, blockdim, ph)
        run_threads(dotprod_tk, blockdim)

In [ ]:
def matmul_2d(m, n, tw=16):
    h,k  = m.shape
    k2,w = n.shape
    assert k==k2, "Size mismatch!"
    output = torch.zeros(h, w, dtype=m.dtype)
    tpb = ns(x=tw,y=tw)
    blocks = ns(x=math.ceil(w/tpb.x), y=math.ceil(h/tpb.y))
    blk_kernel2d_shar(matmul_tiled_bk, blocks, tpb, tw*tw*2,
                      m.flatten(), n.flatten(), output.flatten(),
                      h, w, k, tw=tw)
    return output

In [ ]:
m1s.shape, m2.shape

(torch.Size([8, 784]), torch.Size([784, 10]))

In [ ]:
torch.isclose(matmul_2d(m1s, m2, tw=16), m1s@m2).all()

tensor(True)

### 2d Python kernel threads

In [ ]:
import threading
from threading import Barrier, Thread
from concurrent.futures import ThreadPoolExecutor

In [ ]:
def g(x, sb):
    print(x)
    sb.wait()
    print(-x)
    sb.wait()
    print(x*10)

In [ ]:
num = 3
sb = Barrier(num)
with ThreadPoolExecutor(num) as ex: list(ex.map(lambda i: g(i,sb), range(num)))

0
1
2
-2
-1
0
10
0
20


In [ ]:
def blk_kernel2d_shar(f, blocks, tpb, sh_sz, *args, **kwargs):
    for i0 in range(blocks.y):
        for i1 in range(blocks.x):
            shar = torch.zeros(sh_sz)
            syncb = Barrier(tpb.y*tpb.x)
            threads = [Thread(target=f, args=(ns(x=i1,y=i0), ns(x=p,y=o), tpb, shar, syncb, *args), kwargs=kwargs)
                       for o in range(tpb.y) for p in range(tpb.x)]
            for t in threads: t.start()
            for t in threads: t.join()

In [ ]:
def matmul_tiled_bk(blockidx, threadidx, blockdim, shared, syncb, m, n, out, h, w, k, tw):
    tx,ty = threadidx.x,threadidx.y
    r = blockidx.y*blockdim.y + ty
    c = blockidx.x*blockdim.x + tx

    shar_sz = tw*tw
    ms,ns = shared[:shar_sz],shared[shar_sz:]

    p = 0.
    for ph in range(int(math.ceil(k/tw))):
        ms[ty*tw+tx] = m[ tx + ph*tw + r*k] if r<h and (ph*tw+tx)<k else 0.
        ns[ty*tw+tx] = n[(ty + ph*tw)*w +c] if c<w and (ph*tw+ty)<k else 0.
        syncb.wait()
        for i in range(tw): p += ms[ty*tw+i] * ns[tw*i+tx]
        syncb.wait()

    if (r<h and c<w): out[r*w + c] = p

In [ ]:
def matmul_2d(m, n, tw=16):
    h,k  = m.shape
    k2,w = n.shape
    assert k==k2, "Size mismatch!"
    output = torch.zeros(h, w, dtype=m.dtype)
    tpb = ns(x=tw,y=tw)
    blocks = ns(x=math.ceil(w/tpb.x), y=math.ceil(h/tpb.y))
    blk_kernel2d_shar(matmul_tiled_bk, blocks, tpb, tw*tw*2,
                      m.flatten(), n.flatten(), output.flatten(),
                      h, w, k, tw=tw)
    return output

In [ ]:
%%time
torch.isclose(matmul_2d(m1s, m2, tw=8), m1s@m2).all()

CPU times: user 5.47 s, sys: 4.93 s, total: 10.4 s
Wall time: 4.98 s


tensor(True)

### CUDA

In [ ]:
cuda_src = cuda_begin + r'''
#define TW 16
__global__ void matmul_k(float* m, float* n, float* out, int w, int w, int k, unsigned md_sz, unsigned nd_sz) {
    extern __shared__ float ms_ns[];
    float *ms = (float *) ms_ns;
    float *ns = (float *) ms_ns + md_sz;

    float p = 0;
    for (int ph = 0; ph < ceil(w/(float)TW); ++ph) {
        if ((r < w) && (ph*TW+tx) < w) ms[ty][tx] = M[r*w + ph*TW + tx];
        else ms[ty][tx] = 0.0f;
        if ((ph*TW+ty) < w && c < w) ns[ty][tx] = N[(ph*TW + ty)*w + c];
        else ns[ty][tx] = 0.0f;
        __syncthreads();
        for (int k = 0; k < TW; ++k) p += ms[ty][k] * ns[k][tx];
        __syncthreads();
    }
    if (r < w && c < w) out[r*w + c] = p;
}

torch::Tensor matmul(torch::Tensor m, torch::Tensor n) {
    CHECK_INPUT(m); CHECK_INPUT(n);
    int h = m.size(0);
    int w = n.size(1);
    int k = m.size(1);
    TORCH_CHECK(k==n.size(0), "Size mismatch!");
    auto output = torch::zeros({h, w}, m.options());

    dim3 tpb(16,16);
    dim3 blocks(cdiv(w, tpb.x), cdiv(h, tpb.y));
    matmul_k<<<blocks, tpb>>>(
        m.data_ptr<float>(), n.data_ptr<float>(), output.data_ptr<float>(), h, w, k);
    C10_CUDA_KERNEL_LAUNCH_CHECK();
    return output;
}
'''

In [ ]:
fname = 'matmul'

In [ ]:
def get_sig(fname, src):
    res = re.findall(rf'^(.+\s+{fname}\(.*?\))\s*{{?\s*$', src, re.MULTILINE)
    return res[0]+';' if res else None

In [ ]:
cpp_src = get_sig(fname, cuda_src)
cpp_src

'torch::Tensor matmul(torch::Tensor m, torch::Tensor n);'

In [ ]:
module = load_cuda(cuda_src, cpp_src, [fname])

In [ ]:
m1c,m2c = m1.contiguous().cuda(),m2.contiguous().cuda()

In [ ]:
module.matmul(m1c,m2c).shape

torch.Size([50000, 10])

In [ ]:
torch.isclose(module.matmul(m1c,m2c), m1c@m2c).all()

tensor(True, device='cuda:0')